In [29]:
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.distance import PairwiseDistance
import numpy as np

# Dataset

In [30]:
D = (pd.read_csv("./mnist1.csv"), pd.read_csv("./mnist2.csv"))

In [33]:
D = np.r_[D[0].to_numpy(), D[1].to_numpy()]

In [ ]:
X = D[:, :-1]